In [1]:
import subprocess
import os
import csv
import pandas as pd
from tqdm import tqdm
from datetime import datetime

In [2]:
# function to print elapsed time
def calculate_elapsed_time(start_time, end_time):
    elapsed_time = end_time - start_time
    total_seconds = elapsed_time.total_seconds()
    seconds = int(total_seconds % 60)
    total_minutes = total_seconds // 60
    minutes = int(total_minutes % 60)
    hours = int(total_minutes // 60)
    
    seconds = str(seconds).zfill(2)
    minutes = str(minutes).zfill(2)
    hours = str(hours).zfill(2)

    return str(hours) + ':' + str(minutes) + ':' + str(seconds)

# Initialise

### Read parameter sets to simulate

In [3]:
# read parameters to run
table = pd.read_csv('data/parametersets_to_run.csv')
table

,alphaWalk,alphaBike,alphaCarDriver,alphaCarPassenger,alphaBus,alphaTrain,betaTimeWalk,betaTimeBike,betaTimeCarDriver,betaTimeCarPassenger,betaTimeBus,betaTimeTrain,betaCostCarDriver,betaCostCarPassenger,betaCostBus,betaCostTrain,betaTimeWalkTransport,betaChangesTransport,used_parameter_label,used_parameter_value
0,-10,0,0,0,0,0,-0.04,-0.03,-0.02,-0.02,-0.02,-0.02,-0.15,-0.15,-0.1,-0.1,-0.03,-0.3,alphaWalk,-10
1,-8,0,0,0,0,0,-0.04,-0.03,-0.02,-0.02,-0.02,-0.02,-0.15,-0.15,-0.1,-0.1,-0.03,-0.3,alphaWalk,-8
2,-6,0,0,0,0,0,-0.04,-0.03,-0.02,-0.02,-0.02,-0.02,-0.15,-0.15,-0.1,-0.1,-0.03,-0.3,alphaWalk,-6
3,-4,0,0,0,0,0,-0.04,-0.03,-0.02,-0.02,-0.02,-0.02,-0.15,-0.15,-0.1,-0.1,-0.03,-0.3,alphaWalk,-4
4,-2,0,0,0,0,0,-0.04,-0.03,-0.02,-0.02,-0.02,-0.02,-0.15,-0.15,-0.1,-0.1,-0.03,-0.3,alphaWalk,-2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,0,0,0,0,0,2,-0.04,-0.03,-0.02,-0.02,-0.02,-0.02,-0.15,-0.15,-0.1,-0.1,-0.03,-0.3,alphaTrain,2
92,0,0,0,0,0,4,-0.04,-0.03,-0.02,-0.02,-0.02,-0.02,-0.15,-0.15,-0.1,-0.1,-0.03,-0.3,alphaTrain,4
93,0,0,0,0,0,6,-0.04,-0.03,-0.02,-0.02,-0.02,-0.02,-0.15,-0.15,-0.1,-0.1,-0.03,-0.3,alphaTrain,6
94,0,0,0,0,0,8,-0.04,-0.03,-0.02,-0.02,-0.02,-0.02,-0.15,-0.15,-0.1,-0.1,-0.03,-0.3,alphaTrain,8


### Initialise empty the output dataframe

In [4]:
# Initialise dataframe for mode choice distributions (1 line = 1 distribution = 1 iteration)
df = pd.DataFrame(columns=["CAR_DRIVER", "CAR_PASSENGER", "BIKE", "BUS_TRAM", "TRAIN", "WALK"])

for i in range(0, len(table)):
    df.to_csv("output/iterations/output_proportions_" + str(i) + ".csv", index=False)

# Call Sim2APL iteratively

### Prepare the Java command

In [5]:
# baseline of the Java command
config = "--config src/main/resources/config_DHZW_full.toml"
modeliberal = "--mode-liberal 0.5"
modeconservative = "--mode-conservative 0.5"
output = "-o output/deskrun"
parametersPath = "--parameter_file ../data/parametersets_to_run.csv"

base_cmd = "java -cp target/sim2apl-dhzw-simulation-1.0-SNAPSHOT-jar-with-dependencies.jar main.java.nl.uu.iss.ga.Simulation" + " " + config + " " + modeliberal + " " + modeconservative + " " + output + " " + parametersPath

# set current directory the folder of Sim2APL so I can execute the jar with the correct paths
if(os.path.basename(os.getcwd()) != 'DHZW-simulation_Sim-2APL'):
    new_directory = os.path.join(os.getcwd(), 'DHZW-simulation_Sim-2APL')
    new_directory = new_directory.replace('\\', '/')
    os.chdir(new_directory)

### Main loop to call the simulations

In [6]:
idx_restart = 0 # in case somethings breaks it is easy to continue

In [7]:
time_beginning_all = datetime.now()

for idx, row in tqdm(table.iterrows(), total=table.shape[0]):
    if idx >= idx_restart:
        # repeat each parameter set 5 times
        for iteration in range(0, 5):

            # output file for this iteration
            outputPath = f'--output_file=../output/iterations/output_proportions_{idx}.csv'
            cmd = base_cmd + ' ' + outputPath
            
            # parameterset to use
            arg = f'--parameterset_index={idx+1}'
            full_command = f'{cmd} {arg}'
            
            time_beginning_iteration = datetime.now()
            
            print('parameter number: ' + str(idx) + ' - iteration: ' + str(iteration) + ' START || Time now: ' + time_beginning_iteration.strftime("%H:%M:%S"))
                                    
            # Capture the output of the Java program
            try:
                output = subprocess.check_output(full_command, stderr=subprocess.STDOUT, universal_newlines=True)
            except subprocess.CalledProcessError as e:
                print(f"Java program exited with non-zero return code: {e.returncode}")
                print(f"Error message: {e.output}")
                exit(1)
                
            time_end_iteration = datetime.now()
         
            print('parameter number: ' + str(idx) + ' - iteration: ' + str(iteration) + ' END   || Time now: ' + time_end_iteration.strftime("%H:%M:%S") + ' | Duration iteration: ' + calculate_elapsed_time(time_beginning_iteration, time_end_iteration) + ' | Elapsed from start: ' + calculate_elapsed_time(time_beginning_all, time_end_iteration))

  0%|                                                                                           | 0/96 [00:00<?, ?it/s]

parameter number: 0 - iteration: 0 START || Time now: 12:28:40
parameter number: 0 - iteration: 0 END   || Time now: 12:30:18 | Duration iteration: 00:01:38 | Elapsed from start: 00:01:39
parameter number: 0 - iteration: 1 START || Time now: 12:30:18
parameter number: 0 - iteration: 1 END   || Time now: 12:30:44 | Duration iteration: 00:00:26 | Elapsed from start: 00:02:05
parameter number: 0 - iteration: 2 START || Time now: 12:30:44
parameter number: 0 - iteration: 2 END   || Time now: 12:31:11 | Duration iteration: 00:00:26 | Elapsed from start: 00:02:32
parameter number: 0 - iteration: 3 START || Time now: 12:31:11
parameter number: 0 - iteration: 3 END   || Time now: 12:31:39 | Duration iteration: 00:00:27 | Elapsed from start: 00:02:59
parameter number: 0 - iteration: 4 START || Time now: 12:31:39


  1%|▊                                                                               | 1/96 [03:23<5:22:03, 203.40s/it]

parameter number: 0 - iteration: 4 END   || Time now: 12:32:03 | Duration iteration: 00:00:24 | Elapsed from start: 00:03:24
parameter number: 1 - iteration: 0 START || Time now: 12:32:03
parameter number: 1 - iteration: 0 END   || Time now: 12:32:32 | Duration iteration: 00:00:28 | Elapsed from start: 00:03:52
parameter number: 1 - iteration: 1 START || Time now: 12:32:32
parameter number: 1 - iteration: 1 END   || Time now: 12:33:00 | Duration iteration: 00:00:28 | Elapsed from start: 00:04:21
parameter number: 1 - iteration: 2 START || Time now: 12:33:00
parameter number: 1 - iteration: 2 END   || Time now: 12:33:32 | Duration iteration: 00:00:31 | Elapsed from start: 00:04:52
parameter number: 1 - iteration: 3 START || Time now: 12:33:32
parameter number: 1 - iteration: 3 END   || Time now: 12:33:59 | Duration iteration: 00:00:27 | Elapsed from start: 00:05:20
parameter number: 1 - iteration: 4 START || Time now: 12:33:59


  2%|█▋                                                                              | 2/96 [05:47<4:23:42, 168.32s/it]

...

parameter number: 93 - iteration: 4 END   || Time now: 15:38:17 | Duration iteration: 00:00:24 | Elapsed from start: 03:09:38
parameter number: 94 - iteration: 0 START || Time now: 15:38:17
parameter number: 94 - iteration: 0 END   || Time now: 15:38:42 | Duration iteration: 00:00:24 | Elapsed from start: 03:10:02
parameter number: 94 - iteration: 1 START || Time now: 15:38:42
parameter number: 94 - iteration: 1 END   || Time now: 15:39:05 | Duration iteration: 00:00:22 | Elapsed from start: 03:10:25
parameter number: 94 - iteration: 2 START || Time now: 15:39:05
parameter number: 94 - iteration: 2 END   || Time now: 15:39:29 | Duration iteration: 00:00:24 | Elapsed from start: 03:10:50
parameter number: 94 - iteration: 3 START || Time now: 15:39:29
parameter number: 94 - iteration: 3 END   || Time now: 15:39:52 | Duration iteration: 00:00:23 | Elapsed from start: 03:11:13
parameter number: 94 - iteration: 4 START || Time now: 15:39:52


 99%|██████████████████████████████████████████████████████████████████████████████▏| 95/96 [3:11:34<01:59, 119.15s/it]

parameter number: 94 - iteration: 4 END   || Time now: 15:40:15 | Duration iteration: 00:00:22 | Elapsed from start: 03:11:35
parameter number: 95 - iteration: 0 START || Time now: 15:40:15
parameter number: 95 - iteration: 0 END   || Time now: 15:40:38 | Duration iteration: 00:00:23 | Elapsed from start: 03:11:59
parameter number: 95 - iteration: 1 START || Time now: 15:40:38
parameter number: 95 - iteration: 1 END   || Time now: 15:41:01 | Duration iteration: 00:00:22 | Elapsed from start: 03:12:21
parameter number: 95 - iteration: 2 START || Time now: 15:41:01
parameter number: 95 - iteration: 2 END   || Time now: 15:41:23 | Duration iteration: 00:00:22 | Elapsed from start: 03:12:44
parameter number: 95 - iteration: 3 START || Time now: 15:41:23
parameter number: 95 - iteration: 3 END   || Time now: 15:41:46 | Duration iteration: 00:00:22 | Elapsed from start: 03:13:06
parameter number: 95 - iteration: 4 START || Time now: 15:41:46


100%|███████████████████████████████████████████████████████████████████████████████| 96/96 [3:13:28<00:00, 120.92s/it]

parameter number: 95 - iteration: 4 END   || Time now: 15:42:09 | Duration iteration: 00:00:22 | Elapsed from start: 03:13:29
